openai function with LCEL. Pydantic python library which makes it easier to construct openai functions

Pydanytic

Pydantic is data validation library for python

1. Works with python type annotations. But rather than static type checking, they are actively used at runtime for data validation and conversion

2. Provides built-in methods to serialize/deserialize models to/from JSON, dictionaries etc

3. Langchain leverages Pydantic to create JSON Scheme describing function.

In normal python you would create a class like this:

class user:

    def __init__(self, name:str, age:int, email:str):

        self.name = name

        self.age = age

        self.email = email



using Pydantic:-

from pydantic import BaseModel,

class User(Basemodel):

        name: str

        age:int

        email:Emailstr

In [1]:
import os
from dotenv import load_dotenv
import openai

In [2]:
load_dotenv()
os.environ["OPENAI_API_KEY"] = os.getenv("OPENAI_API_KEY")

In [3]:
from typing import List
from pydantic import BaseModel, Field

In [4]:
#create normal python class


class User:
    def __init__(self,name:str, age:int, email:str):
        self.name = name
        self.age = age
        self.email = email

In [5]:
foo = User(name="Joe",age = 'bar', email="joe@gmail.com")

In [6]:
foo.name

'Joe'

suppose we assign age value to bar, will print bar as output which is wrong. as we want some validation say its integer

In [7]:
foo.age

'bar'

In [8]:
class pUser(BaseModel):
    name:str
    age:int
    email:str

In [9]:
foo_p = pUser(name="Joe",age = 32, email="joe@gmail.com")

In [10]:
foo_p

pUser(name='Joe', age=32, email='joe@gmail.com')

lets try with pydantic try invalid argument for age like before.it does validation

In [11]:
foo_p = pUser(name="Joe",age = 'bar', email="joe@gmail.com")

ValidationError: 1 validation error for pUser
age
  Input should be a valid integer, unable to parse string as an integer [type=int_parsing, input_value='bar', input_type=str]
    For further information visit https://errors.pydantic.dev/2.11/v/int_parsing

In [12]:
class Class(BaseModel):
    students:List[pUser]

In [13]:
obj = Class(
    students= [pUser(name="joe",age=32, email="joe@gmail.com"),
               pUser(name="don",age=28, email="don@gmail.com"),
               pUser(name="doe",age=35, email="doe@gmail.com")]
)

In [14]:
obj

Class(students=[pUser(name='joe', age=32, email='joe@gmail.com'), pUser(name='don', age=28, email='don@gmail.com'), pUser(name='doe', age=35, email='doe@gmail.com')])

Pydanic to create OpenAI function definations

In [15]:
class WeatherSearch(BaseModel):
    """Call this with an airport code to get the weather at that airport"""
    airport_code:str = Field(description="airport code to get weather for")  # Field pydantic module used to set things of various fields.

In [16]:
from langchain.utils.openai_functions import convert_pydantic_to_openai_function  # convert function into json schema

In [17]:
weather_function = convert_pydantic_to_openai_function(WeatherSearch)

/var/folders/s8/qyjb36g92fs3ztdqk120mmkw0000gn/T/ipykernel_19215/1313296718.py:1: LangChainDeprecationWarning: The function `_convert_pydantic_to_openai_function` was deprecated in LangChain 0.1.16 and will be removed in 1.0. Use :meth:`~langchain_core.utils.function_calling.convert_to_openai_function()` instead.
  weather_function = convert_pydantic_to_openai_function(WeatherSearch)


In [18]:
weather_function

{'name': 'WeatherSearch',
 'description': 'Call this with an airport code to get the weather at that airport',
 'parameters': {'properties': {'airport_code': {'description': 'airport code to get weather for',
    'type': 'string'}},
  'required': ['airport_code'],
  'type': 'object'}}

In [19]:
class WeatherSearch1(BaseModel):
    airport_code:str = Field(description="airport code to get weather for")  # Field pydantic module used to set things of various fields.

In [20]:
weather_function1 = convert_pydantic_to_openai_function(WeatherSearch1)

In [21]:
weather_function1

{'name': 'WeatherSearch1',
 'description': '',
 'parameters': {'properties': {'airport_code': {'description': 'airport code to get weather for',
    'type': 'string'}},
  'required': ['airport_code'],
  'type': 'object'}}

In [22]:
class WeatherSearch2(BaseModel):
    """Call this with an airport code to get the weather at that airport"""
    airport_code:str
    
    #descriptions for arguments are optional in Langchain. 

In [23]:
weather_function2 = convert_pydantic_to_openai_function(WeatherSearch2)

In [24]:
weather_function2

{'name': 'WeatherSearch2',
 'description': 'Call this with an airport code to get the weather at that airport',
 'parameters': {'properties': {'airport_code': {'type': 'string'}},
  'required': ['airport_code'],
  'type': 'object'}}

In [25]:
from langchain.chat_models import ChatOpenAI

In [26]:
model = ChatOpenAI()

/var/folders/s8/qyjb36g92fs3ztdqk120mmkw0000gn/T/ipykernel_19215/1603725051.py:1: LangChainDeprecationWarning: The class `ChatOpenAI` was deprecated in LangChain 0.0.10 and will be removed in 1.0. An updated version of the class exists in the :class:`~langchain-openai package and should be used instead. To use it run `pip install -U :class:`~langchain-openai` and import as `from :class:`~langchain_openai import ChatOpenAI``.
  model = ChatOpenAI()


In [27]:
model.invoke("what is the weather in SF today?",functions=[weather_function])

AIMessage(content='', additional_kwargs={'function_call': {'arguments': '{"airport_code":"SFO"}', 'name': 'WeatherSearch'}}, response_metadata={'token_usage': {'completion_tokens': 17, 'prompt_tokens': 70, 'total_tokens': 87, 'completion_tokens_details': {'accepted_prediction_tokens': 0, 'audio_tokens': 0, 'reasoning_tokens': 0, 'rejected_prediction_tokens': 0}, 'prompt_tokens_details': {'audio_tokens': 0, 'cached_tokens': 0}}, 'model_name': 'gpt-3.5-turbo', 'system_fingerprint': None, 'finish_reason': 'function_call', 'logprobs': None}, id='run--7c2a0b0b-b5a7-425e-987e-1857b85d5730-0')

In [28]:
model_with_function = model.bind(functions=[weather_function])

In [29]:
model_with_function.invoke("What is the weather in SF?")

AIMessage(content='', additional_kwargs={'function_call': {'arguments': '{"airport_code":"SFO"}', 'name': 'WeatherSearch'}}, response_metadata={'token_usage': {'completion_tokens': 17, 'prompt_tokens': 69, 'total_tokens': 86, 'completion_tokens_details': {'accepted_prediction_tokens': 0, 'audio_tokens': 0, 'reasoning_tokens': 0, 'rejected_prediction_tokens': 0}, 'prompt_tokens_details': {'audio_tokens': 0, 'cached_tokens': 0}}, 'model_name': 'gpt-3.5-turbo', 'system_fingerprint': None, 'finish_reason': 'function_call', 'logprobs': None}, id='run--80207ac5-a69d-447b-b0f8-c840df9d46aa-0')

In [30]:
model_with_forced_function = model.bind(functions=[weather_function], function_call={"name":"WeatherSearch"})

In [31]:
model_with_forced_function.invoke("What is the weather in sf?")

AIMessage(content='', additional_kwargs={'function_call': {'arguments': '{"airport_code":"SFO"}', 'name': 'WeatherSearch'}}, response_metadata={'token_usage': {'completion_tokens': 7, 'prompt_tokens': 79, 'total_tokens': 86, 'completion_tokens_details': {'accepted_prediction_tokens': 0, 'audio_tokens': 0, 'reasoning_tokens': 0, 'rejected_prediction_tokens': 0}, 'prompt_tokens_details': {'audio_tokens': 0, 'cached_tokens': 0}}, 'model_name': 'gpt-3.5-turbo', 'system_fingerprint': None, 'finish_reason': 'stop', 'logprobs': None}, id='run--042cc8aa-596d-44fa-ad2d-1b69efbe1008-0')

In [32]:
model_with_forced_function.invoke("Hii")

AIMessage(content='', additional_kwargs={'function_call': {'arguments': '{"airport_code":"JFK"}', 'name': 'WeatherSearch'}}, response_metadata={'token_usage': {'completion_tokens': 7, 'prompt_tokens': 74, 'total_tokens': 81, 'completion_tokens_details': {'accepted_prediction_tokens': 0, 'audio_tokens': 0, 'reasoning_tokens': 0, 'rejected_prediction_tokens': 0}, 'prompt_tokens_details': {'audio_tokens': 0, 'cached_tokens': 0}}, 'model_name': 'gpt-3.5-turbo', 'system_fingerprint': None, 'finish_reason': 'stop', 'logprobs': None}, id='run--16b712fb-c524-4215-b5b8-549534abf4b2-0')

here we can see its still forcing up the function to call even though the query is invalid.

Using in a chain

We can use this model bound to function in a chain as we normally world

In [33]:
from langchain.prompts import ChatPromptTemplate

In [34]:
prompt = ChatPromptTemplate.from_messages([
    ("system","You are a helpful assistant"),
    ("user", "{input}")
    ])

In [35]:
chain = prompt | model_with_function

In [36]:
chain.invoke({"input":"What is the weather in SF?"})

AIMessage(content='', additional_kwargs={'function_call': {'arguments': '{"airport_code":"SFO"}', 'name': 'WeatherSearch'}}, response_metadata={'token_usage': {'completion_tokens': 17, 'prompt_tokens': 75, 'total_tokens': 92, 'completion_tokens_details': {'accepted_prediction_tokens': 0, 'audio_tokens': 0, 'reasoning_tokens': 0, 'rejected_prediction_tokens': 0}, 'prompt_tokens_details': {'audio_tokens': 0, 'cached_tokens': 0}}, 'model_name': 'gpt-3.5-turbo', 'system_fingerprint': None, 'finish_reason': 'function_call', 'logprobs': None}, id='run--1f416147-dce1-45d1-a47f-f231c32beb16-0')

Using Multiple Functions

Even better, we can pass a set of functions and let the LLM decide which to use based on the question context.

In [37]:
class ArtistSearch(BaseModel):
    """ Call this to get the name of songs by a particular artist"""
    artist_name: str = Field(description="name of the artist to look up")
    n: int = Field(description="number of results")

In [38]:
functions = [
    convert_pydantic_to_openai_function(WeatherSearch),
    convert_pydantic_to_openai_function(ArtistSearch),
]

In [39]:
model_with_functions = model.bind(functions=functions)

In [40]:
model_with_functions.invoke("What is the weather in SF?")

AIMessage(content='', additional_kwargs={'function_call': {'arguments': '{"airport_code":"SFO"}', 'name': 'WeatherSearch'}}, response_metadata={'token_usage': {'completion_tokens': 17, 'prompt_tokens': 117, 'total_tokens': 134, 'completion_tokens_details': {'accepted_prediction_tokens': 0, 'audio_tokens': 0, 'reasoning_tokens': 0, 'rejected_prediction_tokens': 0}, 'prompt_tokens_details': {'audio_tokens': 0, 'cached_tokens': 0}}, 'model_name': 'gpt-3.5-turbo', 'system_fingerprint': None, 'finish_reason': 'function_call', 'logprobs': None}, id='run--6c5fdec6-fbca-45dd-ac0c-54fcaab99d1e-0')

In [50]:
response = model_with_functions.invoke("What are three songs by Taylor swift?")

In [52]:
response

AIMessage(content='', additional_kwargs={'function_call': {'arguments': '{"artist_name":"Taylor Swift","n":3}', 'name': 'ArtistSearch'}}, response_metadata={'token_usage': {'completion_tokens': 21, 'prompt_tokens': 118, 'total_tokens': 139, 'completion_tokens_details': {'accepted_prediction_tokens': 0, 'audio_tokens': 0, 'reasoning_tokens': 0, 'rejected_prediction_tokens': 0}, 'prompt_tokens_details': {'audio_tokens': 0, 'cached_tokens': 0}}, 'model_name': 'gpt-3.5-turbo', 'system_fingerprint': None, 'finish_reason': 'function_call', 'logprobs': None}, id='run--62d1bd70-67e7-4db2-af82-b57e71be9ed1-0')

In [42]:
response = model_with_functions.invoke("Hii!")

In [47]:
print(response.content)

Hello! How can I assist you today?
